In [ ]:
!playwright install # install playwright
!ollama pull llama3.2

In [ ]:
from IPython.display import Markdown
from openai import NotGiven, OpenAI
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

async def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url using Playwright;
    truncate to 2,000 characters as a sensible limit
    """
    if not url:
        return ""
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="domcontentloaded", timeout=60_000)
        content = await page.content()
        await browser.close()
        soup = BeautifulSoup(content, "html.parser")
        title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            text = soup.body.get_text(separator="\n", strip=True)
        else:
            text = ""
        return (title + "\n\n" + text)[:2_000]


def messages_for(website, **args):
    """
    Return message format
    """
    return [
        {"role": "system", "content": args.get('system_prompt')},
        {"role": "user", "content": args.get('user_prompt_prefix') + website}

    ]


async def summarize(client: OpenAI, **args):
    """
    call OpenAI API, to get content
    """
    model = args.get('model')
    if not model:
        raise NotGiven('model parameter needed')

    website = await fetch_website_contents(args.get('url'))
    response = client.chat.completions.create(
        model=model,
        messages = messages_for(website=website, **args)
    )
    return response.choices[0].message.content


async def display_summary(client: OpenAI, **args):
    """
    display the output nicely 
    """
    summary = await summarize(client, **args)
    display(Markdown(summary))

    
# define system prompt
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

# define user prompt
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.
"""



# create the OpenAI client
llama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# get nicely summarize
urls = ["https://edwarddonner.com", "https://openai.com"]
for url in urls:
    print(f"""
    -------------------------------------------------------------
    webiste:{url}
    -------------------------------------------------------------
    """)
    await display_summary(
        llama,
        model="llama3.2",
        url="", 
        system_prompt=system_prompt, 
        user_prompt_prefix=user_prompt_prefix
    )





